# EXPERIMENTAL NOTEBOOK, DO NOT USE!

In [24]:
# fix imports
import os
import sys

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path)

In [25]:
import matplotlib.pyplot as plt
import numpy as np
from wtracker.eval import *
from wtracker.sim.config import TimingConfig
from wtracker.utils.gui_utils import UserPrompt
from wtracker.utils.path_utils import join_paths

### Timing configuration and log files selection

In [26]:
################################ User Input ################################

# The folder path to the experiment to analyze.
# This folder should contain the log file and time config file.
# If None, the user will be prompted to select a folder.
experiment_folder = "/mnt/c/Users/freid/Desktop/FinalEvaluations/Exp1_config1_CSV"

############################################################################


if experiment_folder is None:
    experiment_folder = UserPrompt.open_directory(title="Select log directory to analyze")

log_file = join_paths(experiment_folder, "bboxes.csv")
time_config_path = join_paths(experiment_folder, "time_config.json")
analysis_save_path = join_paths(experiment_folder, "analyzed.csv")

print("Base directory: ", experiment_folder)
print("Log file: ", log_file)
print("Time config file: ", time_config_path)
print("Analysis save file: ", analysis_save_path)

Base directory:  /mnt/c/Users/freid/Desktop/FinalEvaluations/Exp1_config1_CSV
Log file:  /mnt/c/Users/freid/Desktop/FinalEvaluations/Exp1_config1_CSV/bboxes.csv
Time config file:  /mnt/c/Users/freid/Desktop/FinalEvaluations/Exp1_config1_CSV/time_config.json
Analysis save file:  /mnt/c/Users/freid/Desktop/FinalEvaluations/Exp1_config1_CSV/analyzed.csv


In [27]:
from pprint import pprint

# load the data from the directory
timing_config = TimingConfig.load_json(time_config_path)
analyzer = DataAnalyzer.load(timing_config, log_file)

pprint(timing_config)

TimingConfig(px_per_mm=90,
             mm_per_px=0.011111111111111112,
             frames_per_sec=60,
             ms_per_frame=16.666666666666668,
             imaging_time_ms=200,
             imaging_frame_num=12,
             pred_time_ms=40,
             pred_frame_num=3,
             moving_time_ms=50,
             moving_frame_num=3,
             camera_size_mm=[4, 4],
             camera_size_px=[360, 360],
             micro_size_mm=[0.32, 0.32],
             micro_size_px=[29, 29])


### Analyze log data

The analyzer cleans up the data to our needs, and able to display useful statistics.  
Afterwards, the analyzed data will be passed to a `Plotter` class which draws graphs of the resulting analyzed data.

In [28]:
################################ User Input ################################

# initialize the analyzer on the log data.
analyzer.initialize(period=10)

# remove unwanted frames from the data
analyzer.clean(
    trim_cycles=True,
    imaging_only=True,
    bounds=None,
)

# find anomalies in the resulting data, and remove them if needed
analyzer.calc_anomalies(
    no_preds=True,
    min_bbox_error=np.inf,
    min_dist_error=np.inf,
    min_speed=np.inf,
    min_size=np.inf,
    remove_anomalies=True,  # whether to remove the anomalies from the data, on only detect them
)

# change the units of the time and distance of the resulting analyzed data
#analyzer.change_unit("sec")

############################################################################

,frame,cycle,phase,plt_x,plt_y,cam_x,cam_y,cam_w,cam_h,mic_x,...,worm_deviation_y,worm_deviation,bbox_error,precise_error,speed_anomaly,bbox_error_anomaly,dist_error_anomaly,width_anomaly,height_anomaly,no_pred_anomaly
6152,6152,410,imaging,873,1345,693,1165,360,360,859,...,NaN,NaN,NaN,NaN,False,False,False,False,False,True
6154,6154,410,imaging,873,1345,693,1165,360,360,859,...,NaN,NaN,NaN,NaN,False,False,False,False,False,True
6155,6155,410,imaging,873,1345,693,1165,360,360,859,...,NaN,NaN,NaN,NaN,False,False,False,False,False,True
6211,6211,414,imaging,873,1344,693,1164,360,360,859,...,NaN,NaN,NaN,NaN,False,False,False,False,False,True
6220,6220,414,imaging,873,1344,693,1164,360,360,859,...,NaN,NaN,NaN,NaN,False,False,False,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60098,60098,4006,imaging,93,7,-87,-173,360,360,79,...,NaN,NaN,NaN,NaN,False,False,False,False,False,True
60099,60099,4006,imaging,93,7,-87,-173,360,360,79,...,NaN,NaN,NaN,NaN,False,False,False,False,False,True
60100,60100,4006,imaging,93,7,-87,-173,360,360,79,...,NaN,NaN,NaN,NaN,False,False,False,False,False,True
60101,60101,4006,imaging,93,7,-87,-173,360,360,79,...,NaN,NaN,NaN,NaN,False,False,False,False,False,True


In [29]:
from wtracker.sim.config import ExperimentConfig
from wtracker.utils.bbox_utils import *
from tqdm.auto import tqdm
import cv2 as cv
import matplotlib.pyplot as plt

exp_config = ExperimentConfig.load_json(join_paths(experiment_folder, "exp_config.json"))

H, W = exp_config.orig_resolution

class ImageCreator:
    def __init__(self, analyzer: DataAnalyzer, h: int, w: int) -> None:
        self.image = np.ones((h, w, 3), dtype=np.uint8) * 255

        bboxes = analyzer._orig_data[["wrm_x", "wrm_y", "wrm_w", "wrm_h"]].to_numpy()

        bboxes, is_valid = BoxUtils.discretize(bboxes, (h, w), BoxFormat.XYWH)

        self.is_valid = is_valid
        self.bboxes = bboxes

    def __getitem__(self, idx: int):
        if self.is_valid[idx]:
            bbox = self.bboxes[idx]
            assert bbox[2] > 0 and bbox[3] > 0, f"{idx}"
            img = self.image[bbox[1] : bbox[1] + bbox[3], bbox[0] : bbox[0] + bbox[2]]
            assert img.shape[0] > 0 and img.shape[1] > 0, f"{idx}"
            return img
        else:
            return None
        

def save_images(analyzer: DataAnalyzer, save_folder: str, h: int, w: int):
    image_creator = ImageCreator(analyzer, h, w)
    for i in tqdm(range(len(analyzer._orig_data))):
        image = image_creator[i]
        if image is None:
            continue

        try:
            cv.imwrite(join_paths(save_folder, f"{i:09d}.png"), image)
        except Exception as e:
            print(f"Failed to save image {i:09d}: {e}")

In [30]:
#save_images(analyzer, "images", H, W)

In [31]:
class ImageLoader:
    def __init__(self, folder: str, name_format: str, imread_flags) -> None:
        self.folder = folder
        self.name_format = name_format
        self.imread_flags = imread_flags

    def __getitem__(self, idx: int) -> np.ndarray:
        path = join_paths(self.folder, self.name_format.format(idx))
        return cv.imread(path, flags=self.imread_flags)

In [32]:
""" loader = ImageLoader("images", "{:09d}.png", cv.IMREAD_COLOR)

bg = np.zeros((H, W, 3), dtype=np.uint8)

analyzer.calc_precise_error(loader, background=bg, diff_thresh=10)

print(analyzer.data["precise_error"].mean()) """

' loader = ImageLoader("images", "{:09d}.png", cv.IMREAD_COLOR)\n\nbg = np.zeros((H, W, 3), dtype=np.uint8)\n\nanalyzer.calc_precise_error(loader, background=bg, diff_thresh=10)\n\nprint(analyzer.data["precise_error"].mean()) '

In [33]:
print(analyzer.data["bbox_error"].mean())

0.05778873306417174


In [34]:
# get index of the first true element 

first_true = np.argmax((np.abs(analyzer.data["bbox_error"] - analyzer.data["precise_error"]) > 1e-2))
print(first_true)

print(analyzer.data["bbox_error"].iloc[first_true])
print(analyzer.data["precise_error"].iloc[first_true])

print("frame:", analyzer.data["frame"].iloc[first_true])
print(analyzer.data[["wrm_x", "wrm_y", "wrm_w", "wrm_h"]].iloc[first_true])
print(analyzer.data[["mic_x", "mic_y", "mic_w", "mic_h"]].iloc[first_true])

0
0.0
nan
frame: 15
wrm_x    998.13353
wrm_y    738.91156
wrm_w     13.80742
wrm_h     14.56908
Name: 15, dtype: float64
mic_x    988
mic_y    731
mic_w     29
mic_h     29
Name: 15, dtype: int64


In [35]:
analyzer.print_stats()

Count of Removed Frames: 13477 (20.735%)
Count of No-Pred Frames: 0 (0.0%)
Total Num of Cycles: 4304
Non Perfect Predictions: 36.073%


In [36]:
analyzer.describe(
    columns=["wrm_speed", "bbox_error", "precise_error", "worm_deviation"],
    percentiles=[0.25, 0.5, 0.75, 0.8, 0.9, 0.95, 0.97, 0.98, 0.99],
)

,wrm_speed,bbox_error,precise_error,worm_deviation
count,51427.000000,51518.000000,0.0,51518.000000
mean,0.543876,0.057789,NaN,6.274760
std,0.282611,0.121241,NaN,3.605424
min,0.000570,-0.000000,NaN,0.002450
25%,0.381605,0.000000,NaN,3.792110
50%,0.551570,0.000000,NaN,5.922205
75%,0.692520,0.068115,NaN,8.370817
80%,0.728422,0.105186,NaN,8.998200
90%,0.828114,0.202318,NaN,10.640425
95%,0.939174,0.286930,NaN,12.093912


In [37]:
# save tge analyzed data back to the experiment directory
analyzer.save(analysis_save_path)

### Plotting

Notice that all of below plots accept `condition` as a parameter.
`condition` is expected to be a function of the following signature:

```python
def cond_func1(input_df: pd.DataFrame) -> pd.DataFrame:
    return (input_df["wrm_speed"] > 5) &  (input_df["wrm_speed"] <= 30)
```

In python, such functions can be also declared without an explicit name and declaration, using the following syntax:
(for more information read about lambda functions)

```python
cond_func1 = lambda input_df: (input_df["wrm_speed"] > 5) & (input_df["wrm_speed"] <= 30)
cond_func2 = lambda input_df: input_df["phase"] == "imaging"
```

In [38]:
# print column names of the data
pprint([f"{i}: {col}" for i, col in enumerate(analyzer.column_names())])

['0: frame',
 '1: cycle',
 '2: phase',
 '3: plt_x',
 '4: plt_y',
 '5: cam_x',
 '6: cam_y',
 '7: cam_w',
 '8: cam_h',
 '9: mic_x',
 '10: mic_y',
 '11: mic_w',
 '12: mic_h',
 '13: wrm_x',
 '14: wrm_y',
 '15: wrm_w',
 '16: wrm_h',
 '17: time',
 '18: cycle_step',
 '19: wrm_center_x',
 '20: wrm_center_y',
 '21: mic_center_x',
 '22: mic_center_y',
 '23: wrm_speed_x',
 '24: wrm_speed_y',
 '25: wrm_speed',
 '26: worm_deviation_x',
 '27: worm_deviation_y',
 '28: worm_deviation',
 '29: bbox_error',
 '30: precise_error']


In [39]:
# create the plotter from the analyzed data we created previously

# analyzer.change_unit("sec")

pltr = Plotter([analyzer.data], plot_height=7, palette="bright")

In [40]:
pltr.plot_trajectory()
plt.show()

In [41]:
pltr.plot_head_size()
plt.show()

In [42]:
pltr.plot_speed(
    condition=lambda x: x["wrm_speed"] <= 800,
    aspect=0.5,
)
plt.show()

In [43]:
pltr.plot_cycle_error(
    error_kind="bbox",
    log_wise=True,
    k_depth="proportion",
    outlier_prop=0.01,
)
plt.show()

In [44]:
pltr.plot_cycle_error(
    error_kind="dist",
    log_wise=True,
    k_depth="proportion",
    outlier_prop=0.02,
)
plt.show()

In [45]:
pltr.plot_speed_vs_error(
    error_kind="bbox",
    cycle_wise=True,
    condition=lambda df: (df["wrm_speed"] < 1000) & (df["bbox_error"] > 1e-5),
)
plt.show()

In [46]:
pltr.plot_speed_vs_error(
    error_kind="dist",
    cycle_wise=True,
    condition=lambda df: (df["wrm_speed"] < 1000) & (df["worm_deviation"] < 300),
)
plt.show()